In [1]:
!pip install -r ../data/requirements_sk3_sw.txt

In [16]:
!pip install ipywidgets langchain_huggingface langchain_experimental langchain_openai accelerate -q

In [3]:
import os
import sys
sys.path.append(os.path.abspath("../")) 

In [4]:
# 표준 라이브러리
import os
import sqlite3

# 외부 라이브러리
import numpy as np
import pandas as pd
import torch
from sqlalchemy import text

# HuggingFace Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# LangChain - Core
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.llms import OpenAI

# LangChain - OpenAI
from langchain.chat_models import ChatOpenAI

# LangChain - SQL 및 HuggingFace 연동
from langchain_huggingface import HuggingFacePipeline
from langchain_community.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

os.environ['OPENAI_API_KEY'] = "아_이석원_인턴_강등"

In [5]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain.schema import BaseOutputParser
from langchain_openai import OpenAI
from tools.SQL_create import SQLPostprocessChain
from tools.SQL_execute import SQLExecuteChain

# Step 1. DB 연결
db = SQLDatabase.from_uri("sqlite:///../data/players_position.db")

# Step 1. 자연어 → SQL 변환 프롬프트
sql_prompt = PromptTemplate(
    input_variables=["question", "schema"],
    # input_variables=["question"],
    template="""
당신은 축구와 관련된 데이터베이스에서 입력된 자연어를 SQL구문으로 변경해주는 모델입니다.
사용자가 다음과 같이 질문했습니다:
"{question}"

아래는 축구 선수의 포지션별 데이터 테이블의 스키마입니다:
{schema}
현재_능력과 잠재력은 1~200 사이이고
다른 모든 스탯은 1~20사이입니다.

이 질문에 맞게 문법에 맞는 SQL 쿼리를 작성해 주세요. 
where절에 포지션은 포함하지 말아주고
select절에는 *로 통일해주세요.
sql구문을 생성할 때는 주 스탯의 크기로 정렬해주세요.
출력할 선수는 최대 5명으로 제한해주세요.
하나의 테이블 안에서만 검색을 해주세요
한글로 선수의 이름이나 국가가 입력된다면 가장 유사한 영어 이름으로 매핑해주세요.
축구와 관련있는 질문이 들어올 때만 쿼리를 생성해주세요.
바로 사용할 수 있도록 꼭 문법에 맞는 SQL 구문만 출력해 주세요.
"""
)

sql_chain = LLMChain(llm=ChatOpenAI(temperature=0,model="gpt-4o-mini"), prompt=sql_prompt, output_key="sql_query")
sql_postprocess_chain = SQLPostprocessChain(llm_chain=sql_chain)

# Step 2. SQL 쿼리 실행용 체인 정의
sql_execute_chain = SQLExecuteChain(db_engine=db)
    
# Step 3. 최종 자연어 답변 생성용 체인
final_prompt = PromptTemplate(
    input_variables=["question", "query_result"],
    template="""
아래는 사용자 질문과 이에 대한 쿼리 결과를 바탕으로 친절하게 답변한 예시들입니다.

예시 1:
질문: 질문: 가장 뛰어난 공격수 1명만 추천해줘
쿼리 결과: '이름은 Kylian Mbappé이고, 나이은 23이고, 포지션은 ST이고, 현재_능력은 188이고, 잠재력은 197이고, 국적은 France이고, 소속팀은 Paris Saint-Germain이고, 키은 178이고, 몸무게은 73이고, 왼발_능력은 10이고, 오른발_능력은 20이고, 시장_가치은 347975206이고, 슛은 17이고, 드리블은 18이고, 퍼스트_터치은 18이고, 헤딩은 7이고, 예측력은 17이고, 오프_더_볼은 18이고, 침착성은 18이고, 가속력은 20이고, 체력은 11이고, 민첩성은 16입니다.''
답변: "Kylian Mbappé는 현재 23세의 프랑스 출신 공격수로, Paris Saint-Germain 소속입니다. 그는 현재 능력 188과 잠재력 197을 보유하고 있으며, 스피드와 드리블 능력이 뛰어난 선수입니다. 그의 가속력은 20으로 매우 빠르며, 드리블 능력은 18로 상대 수비수를 쉽게 제치고 돌파할 수 있습니다. 또한, 그의 슛 능력은 17로 골을 넣는 데에도 뛰어난 능력을 발휘합니다. Mbappé는 왼발과 오른발 모두에서 능숙하며, 특히 오른발에서 20의 능력을 보유하고 있습니다. 그의 예측력과 오프 더 볼 능력도 각각 17과 18로 뛰어나, 경기 상황을 잘 읽고 좋은 위치에서 공격을 전개할 수 있습니다. 전반적으로 Kylian Mbappé는 최고의 공격수로서 매우 추천할 만한 선수입니다. 그가 경기를 이끌어 나가는 모습을 보는 것은 정말 흥흥미로운 경험이 될 것입니다."

예시 2:
질문: 패스를 잘하는 미드필더 추천해줘
쿼리 결과: '이름은 Toni Kroos이고, 나이은 32이고, 포지션은 M이고, 현재_능력은 166이고, 잠재력은 176이고, 국적은 Germany이고, 소속팀은 R. Madrid이고, 키은 183이고, 몸무게은 76이고, 왼발_능력은 15이고, 오른발_능력은 20이고, 시장_가치은 37521116이고, 태클은 9이고, 패스은 20이고, 퍼스트_터치은 18이고, 팀워크은 16이고, 판단력은 17이고, 지구력은 12이고, 시야은 18이고, 침착성은 18이고, 활동량은 11입니다.', '이름은 Luka Modrić이고, 나이은 36이고, 포지션은 M이고, 현재_능력은 179이고, 잠재력은 180이고, 국적은 Croatia이고, 소속팀은 R. Madrid이고, 키은 172이고, 몸무게은 67이고, 왼발_능력은 13이고, 오른발_능력은 20이고, 시장_가치은 30331138이고, 태클은 11이고, 패스은 19이고, 퍼스트_터치은 20이고, 팀워크은 17이고, 판단력은 18이고, 지구력은 13이고, 시야은 19이고, 침착성은 18이고, 활동량은 16입니다.', '이름은 Paul Pogba이고, 나이은 29이고, 포지션은 M이고, 현재_능력은 160이고, 잠재력은 176이고, 국적은 France,Guinea이고, 소속팀은 Juventus이고, 키은 191이고, 몸무게은 84이고, 왼발_능력은 13이고, 오른발_능력은 20이고, 시장_가치은 91437501이고, 태클은 11이고, 패스은 19이고, 퍼스트_터치은 18이고, 팀워크은 15이고, 판단력은 12이고, 지구력은 16이고, 시야은 18이고, 침착성은 14이고, 활동량은 8입니다.', '이름은 Kevin De Bruyne이고, 나이은 31이고, 포지션은 M이고, 현재_능력은 189이고, 잠재력은 189이고, 국적은 Belgium이고, 소속팀은 Manchester City이고, 키은 181이고, 몸무게은 68이고, 왼발_능력은 16이고, 오른발_능력은 20이고, 시장_가치은 347975206이고, 태클은 9이고, 패스은 18이고, 퍼스트_터치은 16이고, 팀워크은 14이고, 판단력은 18이고, 지구력은 17이고, 시야은 20이고, 침착성은 15이고, 활동량은 15입니다.', '이름은 Joshua Kimmich이고, 나이은 27이고, 포지션은 M이고, 현재_능력은 178이고, 잠재력은 183이고, 국적은 Germany이고, 소속팀은 FC Bayern München이고, 키은 177이고, 몸무게은 75이고, 왼발_능력은 12이고, 오른발_능력은 20이고, 시장_가치은 120657610이고, 태클은 12이고, 패스은 18이고, 퍼스트_터치은 16이고, 팀워크은 17이고, 판단력은 16이고, 지구력은 20이고, 시야은 18이고, 침착성은 17이고, 활동량은 20입니다.'
답변: 각 선수의 특징을 자세히 설명드릴게요.\n\n1. **Toni Kroos** (32세, 독일, R. Madrid 소속)\n- 패스 능력 20, 시야 18, 판단력 17로 뛰어난 경기 조율 능력을 보유하고 있습니다.\n- 긴 패스와 짧은 패스를 모두 정확하게 구사하며, 팀워크(16)와 침착성(18)이 뛰어나 경기의 중심을 잡는 역할을 탁월하게 수행합니다.\n- 키 183cm, 몸무게 76kg으로 피지컬도 탄탄해 중원에서 안정적인 볼 소유가 가능합니다.\n- 활동량은 다소 적은 편이나, 위치 선정과 경기 이해도가 높아 큰 공백 없이 경기 흐름을 조절합니다.\n\n2. **Luka Modrić** (36세, 크로아티아, R. Madrid 소속)\n- 패스 19, 시야 19, 퍼스트 터치 20, 판단력 18 등 기술적 완성도가 매우 높습니다.\n- 풍부한 경기 경험과 뛰어난 침착성(18)을 바탕으로 경기를 유연하게 이끌며, 미세한 공간을 찾아내는 능력이 탁월합니다.\n- 키 172cm, 몸무게 67kg으로 민첩하고 빠른 움직임이 가능하며, 팀워크(17)와 활동량(16)도 우수해 중원에서 공격과 수비 모두에 기여합니다.\n\n3. **Paul Pogba** (29세, 프랑스/기니, Juventus 소속)\n- 패스 19, 시야 18로 공격 전개에 창의성을 더하는 플레이메이커입니다.\n- 뛰어난 피지컬(키 191cm, 몸무게 84kg)과 지구력(16)을 활용해 중원에서 든든한 존재감을 발휘합니다.\n- 퍼스트 터치(18)와 팀워크(15)가 좋아 동료들과의 연계 플레이가 원활하며, 침착성(14)은 경기 상황에 따라 안정적인 결정력을 돕습니다.\n- 다만 활동량(8)은 상대적으로 낮아 체력 관리가 중요합니다.\n\n4. **Kevin De Bruyne** (31세, 벨기에, Manchester City 소속)\n- 패스 18, 시야 20, 판단력 18로 세계 최고 수준의 플레이메이커 중 한 명입니다.\n- 뛰어난 창의력과 공간 인식으로 결정적인 찬스를 만드는 능력이 탁월하며, 침착성(15)으로 결정적인 순간에도 흔들리지 않습니다.\n- 키 181cm, 몸무게 68kg의 균형 잡힌 체격과 지구력(17), 활동량(15)으로 꾸준한 경기력을 유지합니다.\n- 퍼스트 터치(16)와 팀워크(14)도 뛰어나며, 양발 능력이 좋아 어느 쪽에서도 위협적인 패스가 가능합니다.\n\n5. **Joshua Kimmich** (27세, 독일, FC Bayern München 소속)\n- 패스 18, 시야 18, 팀워크 17로 다재다능한 미드필더입니다.\n- 뛰어난 지구력(20)과 활동량(20)을 바탕으로 공격과 수비 모두에서 활발한 움직임을 보입니다.\n- 판단력 16, 침착성 17로 중요한 순간에 침착한 플레이가 가능하며, 퍼스트 터치(16)로 볼 컨트롤 능력도 뛰어납니다.\n- 키 177cm, 몸무게 75kg의 균형 잡힌 피지컬로 중원에서 안정적인 볼 배급이 가능합니다.\n\n이상으로 패스를 잘하는 미드필더 다섯 명을 추천드립니다. 각 선수마다 강점이 뚜렷하니, 팀 스타일과 전술에 맞게 선택하시면 좋겠습니다.

예시 3:
질문: 나이가 어린 유망주 미드필더를 추천해주세요
쿼리 결과: 이름은 Aurélien Tchouameni이고, 나이은 22이고, 포지션은 MF이고, 현재_능력은 159이고, 잠재력은 180이고, 국적은 France이고, 소속팀은 Real Madrid이고, 키은 187이고, 몸무게은 81이고, 태클은 16이고, 패스은 15이고, 지구력은 17이고, 팀워크은 15이고, 판단력은 15입니다.', '이름은 Eduardo Camavinga이고, 나이은 19이고, 포지션은 MF이고, 현재_능력은 147이고, 잠재력은 180이고, 국적은 France이고, 소속팀은 Real Madrid이고, 키은 182이고, 몸무게은 68이고, 태클은 15이고, 패스은 14이고, 퍼스트_터치은 14이고, 활동량은 16입니다.', '이름은 Jadon Sancho이고, 나이은 22이고, 포지션은 MF이고, 현재_능력은 155이고, 잠재력은 177이고, 국적은 England이고, 소속팀은 Manchester UFC이고, 키은 180이고, 몸무게은 76이고, 패스은 16이고, 퍼스트_터치은 16이고, 시야은 15이고, 침착성은 14입니다.', '이름은 Pedri이고, 나이은 19이고, 포지션은 MF이고, 현재_능력은 159이고, 잠재력은 176이고, 국적은 Spain이고, 소속팀은 Barcelona이고, 키은 174이고, 몸무게은 65이고, 태클은 9이고, 패스은 17이고, 퍼스트_터치은 18이고, 시야은 18입니다.', '이름은 Jude Bellingham이고, 나이은 19이고, 포지션은 MF이고, 현재_능력은 155이고, 잠재력은 175이고, 국적은 England이고, 소속팀은 Borussia Dortmund이고, 키은 186이고, 몸무게은 75이고, 태클은 13이고, 패스은 15이고, 퍼스트_터치은 16이고, 지구력은 16입니다.'",
답변: "추천하는 나이 어린 유망주 미드필더 선수들은 다음과 같습니다:\n\n1. **Aurélien Tchouameni** (22세, 프랑스, Real Madrid 소속)\n   - 현재 능력 159, 잠재력 180으로 뛰어난 성장 가능성을 지닌 선수입니다.\n   - 187cm의 큰 키와 81kg의 탄탄한 체격으로 중원에서 압도적인 피지컬을 보여주며, 태클 능력 16으로 수비적 역할도 훌륭히 수행합니다.\n   - 패스 능력 15와 지구력 17을 갖춰 경기를 폭넓게 커버하고, 팀워크와 판단력 모두 15로 경기 운영 능력이 안정적입니다.\n\n2. **Eduardo Camavinga** (19세, 프랑스, Real Madrid 소속)\n   - 현재 능력 147, 잠재력 180의 기대주로, 빠른 성장세를 보이고 있습니다.\n   - 182cm의 신체 조건과 68kg의 날렵한 몸으로 뛰어난 기동성을 자랑하며, 태클 15로 수비력도 갖췄습니다.\n   - 패스 14, 퍼스트 터치 14의 기술적 능력과 16의 활동량으로 경기 내내 활발히 움직이며 볼 소유와 전개에 기여합니다.\n\n3. **Jadon Sancho** (22세, 잉글랜드, Manchester UFC 소속)\n   - 현재 능력 155, 잠재력 177로 공격적인 재능이 돋보이는 선수입니다.\n   - 180cm, 76kg의 균형 잡힌 신체조건을 지니며, 패스 16과 퍼스트 터치 16으로 공격 전개에 능숙합니다.\n   - 시야 15와 침착성 14로 중요한 순간에 정확한 판단과 플레이를 선보입니다.\n\n4. **Pedri** (19세, 스페인, Barcelona 소속)\n   - 현재 능력 159, 잠재력 176의 매우 재능 있는 미드필더입니다.\n   - 174cm, 65kg으로 비교적 작은 체격이나 태클 9로 수비 기여도 가능합니다.\n   - 패스 17, 퍼스트 터치 18, 시야 18로 공을 다루는 기술과 경기 읽기 능력이 뛰어나 창의적인 플레이 메이커 역할을 수행합니다.\n\n5. **Jude Bellingham** (19세, 잉글랜드, Borussia Dortmund 소속)\n   - 현재 능력 155, 잠재력 175로 유망한 신예입니다.\n   - 186cm의 키와 75kg의 체격으로 신체적 우위를 점하며, 태클 13으로 수비에도 적극적입니다.\n   - 패스 15, 퍼스트 터치 16, 지구력 16으로 경기를 안정적으로 풀어가며, 중원에서 활발히 움직입니다.\n\n이 선수들은 모두 나이가 어리고 성장 가능성이 높은 미드필더로서, 기술적 능력과 경기 이해도가 뛰어나 중원에서 팀에 큰 도움을 줄 수 있는 자원들입니다."

예시 4:
질문: 서울의 날씨는 어떤가요
쿼리 결과: None
답변: "본 챗봇은 축구와 관련된 챗봇입니다. 축구와 관련된 질문만 물어봐주세요."
---

질문: {question}
쿼리 결과: {query_result}
위 정보를 바탕으로 사용자에게 선수의 정보와 특징을 친절하게 답변해 주세요.
선수들의 스탯을 바탕으로 선수별 간단한 설명도 덧붙여주세요.
한국어로 자연스럽게 문장을 생성해주세요.
예시에 대한 출력은 하지 말아주세요.
query_result의 결과가 none이면 "본 챗봇은 축구와 관련된 챗봇입니다. 축구와 관련된 질문만 물어봐주세요." 을 출력해주세요.
출력은 문장형식으로 해주세요.
"""
)

final_answer_chain = LLMChain(llm=ChatOpenAI(model="gpt-4o-mini", temperature=0.7,max_tokens=1024), prompt=final_prompt, output_key="final_answer")

# # Step 4. 전체 체인 묶기
full_chain = SequentialChain(
    chains=[sql_postprocess_chain, sql_execute_chain, final_answer_chain],
    # input_variables=["question", "schema"],
    input_variables=["question"],
    output_variables=["final_answer"],
    verbose=True
)

# Step 5. 실행 예시
# question="나이가 어린 유망주 수비수 추천해주고 비교하여 가장 좋은 선수를 알려주세요"
# question="음바페와 홀란드에 대해 검색해주고 두 선수를 비교해줘"
# question = "10000000 예산 안에서 영입가능한 수비수"
question="플레이 메이커형 선수 좀 추천해줘"
question = "오늘의 점심 메뉴는 뭔가요?"

result = full_chain.invoke({"question":question})
print(result['final_answer'])

C:\Users\Playdata\AppData\Local\Temp\ipykernel_22072\1644244892.py:37: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  sql_chain = LLMChain(llm=ChatOpenAI(temperature=0,model="gpt-4o-mini"), prompt=sql_prompt, output_key="sql_query")
C:\Users\Playdata\AppData\Local\Temp\ipykernel_22072\1644244892.py:37: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  sql_chain = LLMChain(llm=ChatOpenAI(temperature=0,model="gpt-4o-mini"), prompt=sql_prompt, output_key="sql_query")




> Entering new SequentialChain chain...


C:\python_sw\p3\FM\tools\SQL_create.py:54: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = self._llm_chain.run(new_inputs)



> Finished chain.
추천드리는 선수들은 다음과 같습니다:

1. **Kylian Mbappé** (23세, 프랑스, Paris Saint-Germain 소속)
   - 현재 능력 188, 잠재력 197로 매우 유망한 공격수입니다. 뛰어난 가속력(20)과 드리블 능력(18)으로 상대 수비수를 쉽게 제치고 돌파할 수 있습니다. 그의 슛 능력은 17로, 결정적인 순간에 골을 넣는 데 강한 능력을 발휘합니다. 오른발 능력도 20으로 뛰어나, 양발 모두에서 위협적인 플레이를 보여줍니다.

2. **Robert Lewandowski** (33세, 폴란드, Barcelona 소속)
   - 현재 능력 186, 잠재력 190인 레반도프스키는 세계 최고의 공격수 중 한 명입니다. 슛 능력 19로 결정적인 찬스를 골로 연결하는 능력이 뛰어나며, 헤딩 능력도 17로 공중볼 경합에서도 강한 모습을 보입니다. 침착성 17로 중요한 순간에도 흔들리지 않는 결정력을 가지고 있습니다.

3. **Erling Haaland** (22세, 노르웨이/잉글랜드, Manchester City 소속)
   - 현재 능력 185, 잠재력 195로 미래가 기대되는 공격수입니다. 195cm의 큰 키와 88kg의 체격을 바탕으로 헤딩(15)과 파워풀한 슛(18)으로 상대 수비수들에게 두려움을 주고 있습니다. 가속력 17로 빠른 스피드를 자랑하며, 오프 더 볼 능력도 18로 효과적인 위치 선정이 가능합니다.

4. **Karim Benzema** (34세, 프랑스/알제리, R. Madrid 소속)
   - 현재 능력 185, 잠재력 185의 베테랑 공격수입니다. 뛰어난 드리블 능력(16)과 퍼스트 터치(20)로 볼을 안정적으로 다루며, 침착성 18로 중요한 순간에도 차분하게 플레이합니다. 185cm의 키와 82kg의 체격으로 공중볼에서도 강력한 존재감을 발휘합니다.

5. **Harry Kane** (28세, 잉글랜드/아일랜드, Tottenham Hotspur 소속)
   - 현재 능력 183

In [6]:
# 예시: 각 체인별 결과를 별도 변수에 저장하고 출력
# question = "왼발 능력이 좋은 공격수"
# question="나이가 어린 유망주 수비수 추천해주고 비교하여 가장 좋은 선수를 알려주세요"
question = "오늘의 점심 메뉴는 뭔가요?"

# 1단계: 자연어 → SQL 쿼리 변환
sql_output = sql_postprocess_chain.invoke({"question":question} )
sql_output = sql_output['sql_query']
print("Step 1 - SQL query:\n", sql_output)

# 2단계: SQL 쿼리 실행
query_result = sql_execute_chain.run(sql_output)
print("Step 2 - Query result:\n", query_result)

# 3단계: 최종 답변 생성
final_answer = full_chain.run(question=question, query_result=query_result)
print("Step 3 - Final answer:\n", final_answer)

Step 1 - SQL query:
 SELECT * FROM ST
ORDER BY 현재_능력 DESC
LIMIT 5
Step 2 - Query result:
 이름은 Kylian Mbappé이고, 나이은 23이고, 포지션은 ST이고, 현재_능력은 188이고, 잠재력은 197이고, 국적은 France이고, 소속팀은 Paris Saint-Germain이고, 키은 178이고, 몸무게은 73이고, 왼발_능력은 10이고, 오른발_능력은 20이고, 시장_가치은 347975206이고, 슛은 17이고, 드리블은 18이고, 퍼스트_터치은 18이고, 헤딩은 7이고, 예측력은 17이고, 오프_더_볼은 18이고, 침착성은 18이고, 가속력은 20이고, 체력은 11이고, 민첩성은 16입니다.
이름은 Robert Lewandowski이고, 나이은 33이고, 포지션은 ST이고, 현재_능력은 186이고, 잠재력은 190이고, 국적은 Poland이고, 소속팀은 Barcelona이고, 키은 185이고, 몸무게은 81이고, 왼발_능력은 13이고, 오른발_능력은 20이고, 시장_가치은 347975206이고, 슛은 19이고, 드리블은 13이고, 퍼스트_터치은 18이고, 헤딩은 17이고, 예측력은 19이고, 오프_더_볼은 19이고, 침착성은 17이고, 가속력은 14이고, 체력은 16이고, 민첩성은 13입니다.
이름은 Erling Haaland이고, 나이은 22이고, 포지션은 ST이고, 현재_능력은 185이고, 잠재력은 195이고, 국적은 Norway,England이고, 소속팀은 Manchester City이고, 키은 195이고, 몸무게은 88이고, 왼발_능력은 20이고, 오른발_능력은 11이고, 시장_가치은 347975206이고, 슛은 18이고, 드리블은 14이고, 퍼스트_터치은 16이고, 헤딩은 15이고, 예측력은 18이고, 오프_더_볼은 18이고, 침착성은 18이고, 가속력은 17이고, 체력은 17이고, 민첩성은 16입니다.
이름은 Karim Benzema이고, 나이은 34이고, 포지션은 ST